# 양자AI 미분 가능 프로그래밍

# 1. 양자컴퓨팅과 양자인공지능의 원리

In [1]:
using Symbolics
using Printf
using Statistics

## One Qubit CIrcuit

## 1.1 양자비트 상태 표현과 측정

### 켓 벡터

In [2]:
e₀ = [1.; 0.]
e₁ = [0.; 1.]

2-element Vector{Float64}:
 0.0
 1.0

In [3]:
@printf("<0|0> = %.1f, <1|1> = %.1f", e₀' * e₀, e₁' * e₁)

<0|0> = 1.0, <1|1> = 1.0

In [4]:
@printf("<0|1> = %.1f, <0|1> = %.1f", e₀' * e₁, e₁' * e₀)

<0|1> = 0.0, <0|1> = 0.0

In [5]:
@variables α β
q_cat = α*e₀ + β*e₁ 

2-element Vector{Num}:
 α
 β

### 양자상태측정

In [6]:
function measure(qubit_state, N::Int = 1)
    m_array = zeros(Int, N)
    for n in 1:N
        if rand() < abs(qubit_state[1])^2
            m_array[n] = 0
        else
            m_array[n] = 1
        end
    end
    ifelse(N==1, m_array[1],  m_array)
end 

measure (generic function with 2 methods)

In [7]:
sprint_array(X) = join(map(x->@sprintf("%d", x), X), ", ")
@printf("M(|0>) = %d\n", measure([1.;0.]))
println("M(|0>) = [", sprint_array(measure([1.;0.], 10)), "]")
@printf("M(|1>) = %d\n", measure([0.;1.]))
println("M(|1>) = [", sprint_array(measure([0.;1.], 10)), "]")

M(|0>) = 0
M(|0>) = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
M(|1>) = 1
M(|1>) = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [8]:
println("M(|+>) = M([1,1]/sqrt(2)) = [", sprint_array(measure([1.;1.]/sqrt(2), 10)), "]")
println("M(|->) = M([1,-1]/sqrt(2)) = [", sprint_array(measure([1.;-1.]/sqrt(2), 10)), "]")

M(|+>) = M([1,1]/sqrt(2)) = [0, 0, 0, 0, 1, 1, 0, 1, 1, 1]
M(|->) = M([1,-1]/sqrt(2)) = [0, 1, 1, 1, 1, 0, 1, 0, 0, 0]


#### 평균 구하기  

In [9]:
N = 100
@printf("Average of %d times measurements of |0> is %.2f\n", 
    N, mean(measure([1.;0.], N)))
@printf("Average of %d times measurements of |1> is %.2f\n", 
    N, mean(measure([0.;1.], N)))
@printf("Average of %d times measurements of |+> is %.2f\n", 
    N, mean(measure([1.;1.]/sqrt(2), N)))
@printf("Average of %d times measurements of |-> is %.2f\n", 
    N, mean(measure([1.;-1.]/sqrt(2), N)))

Average of 100 times measurements of |0> is 0.00
Average of 100 times measurements of |1> is 1.00
Average of 100 times measurements of |+> is 0.57
Average of 100 times measurements of |-> is 0.50


## 1.2 단일 양자비트 연산자

### 양자 반전 연산자

In [10]:
X = e₁ * e₀' + e₀ * e₁'

2×2 Matrix{Float64}:
 0.0  1.0
 1.0  0.0

In [24]:
println(X * e₀ == e₁, ", ", X * e₁ == e₀)

true, true


In [25]:
display(X * q_cat)

2-element Vector{Num}:
 β
 α

### 양자중첩 연산자

In [27]:
H = [1. 1.; 1. -1.] / sqrt(2)

2×2 Matrix{Float64}:
 0.707107   0.707107
 0.707107  -0.707107

In [31]:
e₊ = H * e₀
e₋ = H * e₁
e₊, e₋ 

([0.7071067811865475, 0.7071067811865475], [0.7071067811865475, -0.7071067811865475])

In [33]:
println(e₊ == (e₀ + e₁) / sqrt(2), ", ", e₋ == (e₀ - e₁) / sqrt(2))

true, true


In [37]:
H*q_cat * sqrt(2)

2-element Vector{Num}:
 α + β
 α - β

### 파울리 연산자

In [63]:
Y = [0 -1.0im; 1.0im 0]
Z = [1. 0; 0 -1.] 
display(Y)
display(Z)

2×2 Matrix{ComplexF64}:
 0.0+0.0im  -0.0-1.0im
 0.0+1.0im   0.0+0.0im

2×2 Matrix{Float64}:
 1.0   0.0
 0.0  -1.0

In [66]:
display(Y * q_cat)
display(Z * q_cat)

2-element Vector{Complex{Num}}:
 -im*β
  im*α

2-element Vector{Num}:
  α
 -β

## 1.3 다중 양자비트 표현 및 연산

In [ ]:
Qubit1 = a[1] * e[:,1] + b[1] * e[:,2]
Qubit2 = a[2] * e[:,1] + b[2] * e[:,2]
Qubits = [Qubit1 Qubit2]

In [10]:
e = [1. 0.; 0. 1.]
Qubit1 = a[1] * e[:,1] + b[1] * e[:,2]

2-element Vector{Num}:
 a[1]
 b[1]

In [11]:
# Hadamard transform
H = [1 1; 1 -1] / sqrt(2)

2×2 Matrix{Float64}:
 0.707107   0.707107
 0.707107  -0.707107

In [ ]:
circuit = H * Qubit1 

LoadError: UndefVarError: Qubit1 not defined

In [13]:
circuit = H * circuit

2-element Vector{Num}:
 0.9999999999999998a[1]
 0.9999999999999998b[1]

In [14]:
H*H

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

## Two Qubit Circuit

In [24]:
Qubit1 = a[1] * e[:,1] + b[1] * e[:,2]
Qubit2 = a[2] * e[:,1] + b[2] * e[:,2]
Qubits = [Qubit1 Qubit2]

2×2 Matrix{Num}:
 a[1]  a[2]
 b[1]  b[2]

In [29]:
Qubit12 = kron(Qubits[:,1], Qubits[:,2])

4-element Vector{Num}:
 a[1]*a[2]
 a[1]*b[2]
 a[2]*b[1]
 b[1]*b[2]

In [30]:
CNOT = [1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0]

4×4 Matrix{Int64}:
 1  0  0  0
 0  1  0  0
 0  0  0  1
 0  0  1  0

In [31]:
cur = CNOT * Qubit12

4-element Vector{Num}:
 a[1]*a[2]
 a[1]*b[2]
 b[1]*b[2]
 a[2]*b[1]

## Bell Circuit

In [49]:
Qubit1 = [1.; 0.] # |0>
Qubit2 = [1.; 0.] # |0>

2-element Vector{Float64}:
 1.0
 0.0

In [50]:
# load at circuit
chain = [Qubit1 Qubit2]

2×2 Matrix{Float64}:
 1.0  1.0
 0.0  0.0

In [58]:
chain[:,1] = H * chain[:,1]

2-element Vector{Float64}:
 0.7071067811865474
 0.7071067811865474

In [68]:
circuit = kron(chain[:,1], chain[:,2])

4-element Vector{Float64}:
 0.7071067811865474
 0.0
 0.7071067811865474
 0.0

In [69]:
circuit = CNOT * circuit

4-element Vector{Float64}:
 0.7071067811865474
 0.0
 0.0
 0.7071067811865474